In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import statsmodels.api as stm
import statsmodels.stats.weightstats as ws
from sklearn.linear_model import LinearRegression
from sklearn import metrics as mt
import math

In [25]:
from pathlib import Path
# `path.parents[1]` is the same as `path.parent.parent`
path= Path().resolve().parent
pathdata = str(path) +"/Data/"
pathdata

'/home/hadoop/Git_Ia/CegepSteFoy_IA/04_Eda/Data/'

In [26]:
donnee = pd.read_csv(pathdata+ "Fichiers12/Immobilierv0r1.csv")
stats=donnee.describe()
dimensions=donnee.shape
nomsvariables = pd.DataFrame(donnee.columns)

In [27]:
stats

,Âge de la maison,Distance station transport,Nombre de dépanneurs à proximité,Latitude,Longitude,Prix par pied carré
count,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000
mean,17.712560,1083.885689,4.094203,24.969030,121.533361,37.980193
std,11.392485,1262.109595,2.945562,0.012410,0.015347,13.606488
min,0.000000,23.382840,0.000000,24.932070,121.473530,7.600000
25%,9.025000,289.324800,1.000000,24.963000,121.528085,27.700000
50%,16.100000,492.231300,4.000000,24.971100,121.538630,38.450000
75%,28.150000,1454.279000,6.000000,24.977455,121.543305,46.600000
max,43.800000,6488.021000,10.000000,25.014590,121.566270,117.500000


In [29]:
donnee.head()

,Âge de la maison,Distance station transport,Nombre de dépanneurs à proximité,Latitude,Longitude,Prix par pied carré
0,32.0,84.87882,10,24.98298,121.54024,37.9
1,19.5,306.59470,9,24.98034,121.53951,42.2
2,13.3,561.98450,5,24.98746,121.54391,47.3
3,13.3,561.98450,5,24.98746,121.54391,54.8
4,5.0,390.56840,5,24.97937,121.54245,43.1


In [43]:
corr = donnee.corr()

In [37]:
Ystand = donnee['Prix par pied carré']
Xstand = donnee['Âge de la maison']
Xstand = stm.add_constant(Xstand)
modele = stm.OLS(Ystand, Xstand).fit()
print(modele.summary())

                             OLS Regression Results                            
Dep. Variable:     Prix par pied carré   R-squared:                       0.044
Model:                             OLS   Adj. R-squared:                  0.042
Method:                  Least Squares   F-statistic:                     19.11
Date:                 Thu, 10 Nov 2022   Prob (F-statistic):           1.56e-05
Time:                         20:14:07   Log-Likelihood:                -1658.3
No. Observations:                  414   AIC:                             3321.
Df Residuals:                      412   BIC:                             3329.
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               42.4347   

In [40]:
modele.predict(Xstand.assign(const=1))
type(modele)

statsmodels.regression.linear_model.RegressionResultsWrapper

In [41]:
## https://www.statology.org/confidence-intervals-python/
sts.norm.interval(alpha=0.95, loc=np.mean(Ystand), scale=sts.sem(Ystand))

(36.6695214415256, 39.290865031904346)

In [44]:
from sklearn.preprocessing import MinMaxScaler
scalerX = MinMaxScaler()
Xnorm = scalerX.fit_transform(X.values)
scalerY = MinMaxScaler()
Ynorm = scalerY.fit_transform(Y.values.reshape(-1,1))
Xnorm=pd.DataFrame(Xnorm)
Xnorm.columns=X.columns
Ynorm=pd.DataFrame(Ynorm)
Ynorm.columns=['Prix par pied carré']

In [48]:
## Normalisation
modele = stm.OLS(Ynorm, Xnorm.assign(const=1))
resultats = modele.fit()
Y_chap = resultats.predict(Xnorm.assign(const=1))
resultats.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     Prix par pied carré   R-squared:                       0.972
Model:                             OLS   Adj. R-squared:                  0.971
Method:                  Least Squares   F-statistic:                     989.2
Date:                 Thu, 10 Nov 2022   Prob (F-statistic):               0.00
Time:                         20:39:52   Log-Likelihood:                 1056.3
No. Observations:                  507   AIC:                            -2077.
Df Residuals:                      489   BIC:                            -2001.
Df Model:                           17                                         
Covariance Type:             nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
coude_dia (cm)          0.0125      0.018      0.694      0.488      -0.023       0.048
poignet_dia (cm)        0.0045      0.017      0.265      0.791      -0.029       0.038
genou_dia (cm)          0.0660      0.017      3.913      0.000       0.033       0.099
cheville_dia (cm)       0.0167      0.017      1.005      0.315      -0.016       0.049
épaule_tour (cm)        0.0492      0.021      2.383      0.018       0.009       0.090
pec_tour (cm)           0.1077      0.023      4.635      0.000       0.062       0.153
taille_tour (cm)        0.3041      0.022     14.028      0.000       0.262       0.347
abdomen_tour (cm)      -0.0259      0.020     -1.311      0.191      -0.065       0.013
hanche_tour (cm)        0.2498      0.024     10.448      0.000       0.203       0.297
bicept_tour (cm)        0.0553      0.023      2.406      0.017       0.010       0.100
avantbras_tour (cm)     0.1072      0.026      4.184      0.000       0.057       0.157
genou_tour (cm)         0.1134      0.022      5.264      0.000       0.071       0.156
cheville_tour (cm)      0.0342      0.018      1.948      0.052      -0.000       0.069
poignet_tour (cm)      -0.0393      0.019     -2.023      0.044      -0.078      -0.001
Âge                    -0.0430      0.008     -5.093      0.000      -0.060      -0.026
Grandeur (cm)           0.2043      0.012     16.390      0.000       0.180       0.229
Genre_Femme             0.0326      0.007      4.469      0.000       0.018       0.047
const                  -0.1494      0.010    -14.486      0.000      -0.170      -0.129
==============================================================================
Omnibus:                       31.783   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               68.775
Skew:                           0.347   Prob(JB):                     1.16e-15
Kurtosis:                       4.666   Cond. No.                         49.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""